In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [7]:
data = pd.read_csv("C:\\Users\\Apoor\\Downloads\\train.csv.zip")

In [8]:
data.drop(['Accident_ID'],axis=1,inplace=True)
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Safety_Score,10000.0,41.876406,16.138072,0.000000,30.593607,41.278539,52.511416,100.000000
Days_Since_Inspection,10000.0,12.931100,3.539803,1.000000,11.000000,13.000000,15.000000,23.000000
Total_Safety_Complaints,10000.0,6.564300,6.971982,0.000000,2.000000,4.000000,9.000000,54.000000
Control_Metric,10000.0,65.145324,11.882934,0.000000,56.927985,65.587967,73.336372,100.000000
Turbulence_In_gforces,10000.0,0.381495,0.121301,0.134000,0.293665,0.365879,0.451346,0.882648
Cabin_Temperature,10000.0,79.969331,2.759739,74.740000,77.960000,79.540000,81.560000,97.510000
Accident_Type_Code,10000.0,3.814900,1.902577,1.000000,2.000000,4.000000,5.000000,7.000000
Max_Elevation,10000.0,32001.803282,9431.995196,831.695553,25757.636908,32060.336419,38380.641513,64297.651218
Violations,10000.0,2.012200,1.039980,0.000000,1.000000,2.000000,3.000000,5.000000
Adverse_Weather_Metric,10000.0,0.255635,0.381128,0.000316,0.012063,0.074467,0.354059,2.365378


In [9]:
dataNumericals = pd.DataFrame(data, columns =data.columns[data.dtypes == 'float64'])
dataNumericals.head()

,Safety_Score,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Max_Elevation,Adverse_Weather_Metric
0,49.223744,71.285324,0.272118,78.04,31335.476824,0.424352
1,62.465753,72.288058,0.423939,84.54,26024.711057,0.352350
2,63.059361,66.362808,0.322604,78.86,39269.053927,0.003364
3,48.082192,74.703737,0.337029,81.79,42771.499200,0.211728
4,26.484018,47.948952,0.541140,77.16,35509.228515,0.176883


In [10]:
dataNumericals=dataNumericals.apply(zscore)

In [11]:
floats = dataNumericals.columns[dataNumericals.dtypes == 'float64']
for columns in floats:
    indexNames_larger = dataNumericals[dataNumericals[columns]>3].index
    indexNames_lesser = dataNumericals[dataNumericals[columns]<-3].index
    
    dataNumericals.drop(indexNames_larger , inplace=True)
    dataNumericals.drop(indexNames_lesser , inplace=True)
    data.drop(indexNames_larger , inplace=True)
    data.drop(indexNames_lesser , inplace=True)

In [12]:
dataNumericals.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9507 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Safety_Score            9507 non-null   float64
 1   Control_Metric          9507 non-null   float64
 2   Turbulence_In_gforces   9507 non-null   float64
 3   Cabin_Temperature       9507 non-null   float64
 4   Max_Elevation           9507 non-null   float64
 5   Adverse_Weather_Metric  9507 non-null   float64
dtypes: float64(6)
memory usage: 519.9 KB


In [13]:
data.drop(data.columns[data.dtypes == 'float64'],axis=1,inplace=True)

In [14]:
for column in dataNumericals.columns:
    data[column]=dataNumericals[column]

In [15]:
data['Severity'].unique()

array(['Minor_Damage_And_Injuries', 'Significant_Damage_And_Fatalities',
       'Significant_Damage_And_Serious_Injuries',
       'Highly_Fatal_And_Damaging'], dtype=object)

In [16]:
encoder=LabelEncoder()
data['Severity']=encoder.fit_transform(data['Severity'])

In [17]:
data['Total_Safety_Complaints'] = np.power(2, data['Total_Safety_Complaints'])
data['Days_Since_Inspection'] = np.power(2, data['Days_Since_Inspection'])
data['Safety_Score'] = np.power(2, data['Safety_Score'])

In [18]:
from sklearn.model_selection import train_test_split

X = data.drop("Severity", axis=1)
y = data["Severity"] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (7605, 10)
X_test shape: (1902, 10)
y_train shape: (7605,)
y_test shape: (1902,)


In [20]:
numerical_columns = ['Safety_Score', 'Control_Metric', 'Turbulence_In_gforces', 'Cabin_Temperature', 'Max_Elevation']
numerical_data = data[numerical_columns]
scaler = StandardScaler()
standardized_data = scaler.fit_transform(numerical_data)
standardized_df = pd.DataFrame(standardized_data, columns=numerical_columns)

print(standardized_df.head())

   Safety_Score  Control_Metric  Turbulence_In_gforces  Cabin_Temperature  \
0      0.105764        0.508013              -0.916317          -0.705806   
1      1.181296        0.594758               0.411614           1.780757   
2      1.245327        0.082177              -0.474725          -0.392117   
3      0.038589        0.803733              -0.348556           0.728749   
4     -0.769495       -1.510767               1.436738          -1.042449   

   Max_Elevation  
0      -0.037005  
1      -0.615335  
2       0.826944  
3       1.208353  
4       0.417508  


In [21]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=42)

model.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [22]:
Days_Since_Inspection = int(input("Days Since Inspection (1-23): "))
Total_Safety_Complaints = int(input("Total Safety Complaints (0-54): "))
Accident_Type_Code = int(input("Accident Type Code (1-7): "))
Violations = int(input("Number of Violations (0-5): "))
Safety_Score = float(input("Safety Score (0-100): "))
Control_Metric = float(input("Control Metric (0-100): "))
Turbulence_In_gforces = float(input("Turbulence In g-forces (0.13-0.88): "))
Cabin_Temperature = float(input("Cabin Temperature (74.7-97.5): "))
Max_Elevation = float(input("Max Elevation (832-64.3k): "))
Adverse_Weather_Metric = float(input("Adverse Weather Metric (0-2.37): "))


user_input = [Days_Since_Inspection,
              Total_Safety_Complaints,
              Accident_Type_Code,
              Violations,
              Safety_Score,
              Control_Metric,
              Turbulence_In_gforces,
              Cabin_Temperature,
              Max_Elevation,
              Adverse_Weather_Metric]

user_input_df = pd.DataFrame([user_input], columns=X_train.columns)


predicted_severity = model.predict(user_input_df)


severity_mapping = {0: 'Minor', 1: 'Moderate', 2: 'Major', 3: 'High'}
predicted_severity = severity_mapping[predicted_severity[0]]


print(f"Predicted Severity: {predicted_severity}")

Predicted Severity: Major


In [23]:
X_train_noisy = X_train + np.random.normal(0, 0.1, X_train.shape)
model.fit(X_train_noisy, y_train)
accuracy = model.score(X_test, y_test)
print(f"Accuracy on test set: {accuracy:.2f}")

Accuracy on test set: 0.87


In [24]:
import pickle

In [25]:
with open('trained_model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

In [26]:
# Load the trained model
with open('trained_model.pkl', 'rb') as model_file:
    loaded_model = pickle.load(model_file)

In [27]:
Days_Since_Inspection = int(input("Days Since Inspection (1-23): "))
Total_Safety_Complaints = int(input("Total Safety Complaints (0-54): "))
Accident_Type_Code = int(input("Accident Type Code (1-7): "))
Violations = int(input("Number of Violations (0-5): "))
Safety_Score = float(input("Safety Score (0-100): "))
Control_Metric = float(input("Control Metric (0-100): "))
Turbulence_In_gforces = float(input("Turbulence In g-forces (0.13-0.88): "))
Cabin_Temperature = float(input("Cabin Temperature (74.7-97.5): "))
Max_Elevation = float(input("Max Elevation (832-64.3k): "))
Adverse_Weather_Metric = float(input("Adverse Weather Metric (0-2.37): "))


user_input = [Days_Since_Inspection,
              Total_Safety_Complaints,
              Accident_Type_Code,
              Violations,
              Safety_Score,
              Control_Metric,
              Turbulence_In_gforces,
              Cabin_Temperature,
              Max_Elevation,
              Adverse_Weather_Metric]

user_input_df = pd.DataFrame([user_input], columns=X_train.columns)


predicted_severity = loaded_model.predict(user_input_df)


severity_mapping = {0: 'Minor', 1: 'Moderate', 2: 'Major', 3: 'High'}
predicted_severity = severity_mapping[predicted_severity[0]]


print(f"Predicted Severity: {predicted_severity}")

Predicted Severity: High
